In [15]:
!pip install wbdata

  Using cached https://files.pythonhosted.org/packages/76/87/a04308d9c80829c3b9c4eefbda6f4c849f2c8afa89917906c63ca6d611a5/wbdata-0.3.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c4/f4/770ae9385990f5a19a91431163d262182d3203662ea2b5739d0fcfc080f1/tabulate-0.8.7-py3-none-any.whl
You are using pip version 9.0.3, however version 20.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
import pandas as pd
import numpy as np
import warnings
import xarray
import cfe
import wbdata as wb
import plotly.express as px

from ipywidgets import interact, interactive, Dropdown, IntSlider
import ipywidgets as widgets

In [6]:
HH_and_Expenditure_Sheet = {'P3 HH/E Characteristics':'1RpMK9oo6knSG_eJMTdUJltq937WlIqAK0IE5inTLvfQ',
        }
Sheets = HH_and_Expenditure_Sheet

def dataframe_from_gspreadsheet(sheet_name, key):
    url = 'https://docs.google.com/spreadsheets/d/{key}/gviz/tq?tqx=out:csv&sheet={sheet_name}&headers=1'.format(
        key=key, sheet_name=sheet_name.replace(' ', '%20'))
    df = pd.read_csv(url)
    return df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

x = dataframe_from_gspreadsheet("Nigeria Expenditures", HH_and_Expenditure_Sheet['P3 HH/E Characteristics'])

z = dataframe_from_gspreadsheet("Nigeria HH Characteristics", HH_and_Expenditure_Sheet['P3 HH/E Characteristics'])


z['m'] = 1
x['m'] = 1

# Take logs of expenditures; call this y
y = np.log(x.replace(0,np.nan).set_index(['j','t','m']))

z.set_index(['j','t','m'],inplace=True)
x.head()

,t,j,"(Cocoyam, Spinach, etc)",Agricultural eggs,Animal fat,Apples,Avocado pear,Baby milk powder,Bananas,Beef,...,Tea,Tomato puree(canned),Tomatoes,Watermelon,Wheat flour,White beans,Wild game meat,Yam flour,Yam-roots,m
0,2010,10001,NaN,280.0,NaN,NaN,NaN,NaN,200.0,500.0,...,NaN,150.0,150.0,NaN,NaN,600.0,NaN,NaN,1500.0,1
1,2010,10002,NaN,280.0,NaN,NaN,NaN,NaN,180.0,1200.0,...,140.0,240.0,120.0,NaN,NaN,400.0,NaN,NaN,1200.0,1
2,2010,10003,NaN,180.0,NaN,NaN,NaN,NaN,100.0,500.0,...,60.0,90.0,100.0,NaN,NaN,100.0,NaN,NaN,400.0,1
3,2010,10004,NaN,180.0,NaN,NaN,NaN,NaN,100.0,500.0,...,30.0,60.0,100.0,NaN,NaN,100.0,NaN,NaN,400.0,1
4,2010,10006,NaN,NaN,NaN,NaN,NaN,NaN,300.0,300.0,...,650.0,NaN,400.0,NaN,NaN,NaN,NaN,NaN,400.0,1


In [8]:
x_2010 = x[0:4826]
x_2012 = x[4827:9585]
x_2015 = x[9586:14165]
x_2018 = x[14166: 19142]

y_2010 = y[0:4826]
y_2012 = y[4827:9585]
y_2015 = y[9586:14165]
y_2018 = y[14166: 19142]

In [9]:
#household size in ln(sum_of_columns)
z.head()

,,,M 0-3,M 4-8,M 9-13,M 14-18,M 19-30,M 31-50,M 51+,F 0-3,F 4-8,F 9-13,F 14-18,F 19-30,F 31-50,F 51+,log_Hsize
j,t,m,,,,,,,,,,,,,,,
10001,2010,1,0,0,0,0,1,2,0,1,0,0,0,1,2,0,1.945910
10002,2010,1,0,0,1,1,1,1,0,0,0,0,0,2,1,0,1.945910
10003,2010,1,0,1,1,1,0,0,1,0,0,1,0,0,1,0,1.791759
10004,2010,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1.098612
10006,2010,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1.098612


In [10]:
result = cfe.Result(y=y,z=z)
result

<xarray.Result>
Dimensions:           (i: 42, j: 8522, k: 15, m: 1, t: 4)
Coordinates:
  * j                 (j) int64 10001 10002 10003 10004 ... 379151 379154 379155
  * m                 (m) int64 1
  * t                 (t) int64 2010 2012 2015 2018
  * i                 (i) <U36 '(Cocoyam, Spinach, etc)' ... 'Yam-roots'
  * k                 (k) <U9 'M 0-3' 'M 4-8' 'M 9-13' ... 'F 51+' 'log_Hsize'
Data variables:
    alpha             object None
    beta              object None
    delta             object None
    prices            object None
    characteristics   (k, j, t, m) float64 0.0 1.0 1.0 1.0 ... nan nan nan 1.386
    loglambdas        object None
    a                 object None
    yhat              object None
    ce                object None
    cehat             object None
    e                 object None
    delta_covariance  object None
    se_delta          object None
    se_beta           object None
    se_alpha          object None
    se_a             

In [11]:
result.get_reduced_form()

In [12]:
result.delta.to_dataframe().unstack('k')

delta                                \
k                                        F 0-3   F 14-18   F 19-30   F 31-50   
i                                                                              
(Cocoyam, Spinach, etc)              -0.060970 -0.011346  0.056436  0.082428   
Agricultural eggs                    -0.025314 -0.005658  0.028274  0.084036   
Bananas                               0.011043  0.021719  0.055049  0.084422   
Beef                                 -0.013914  0.048407  0.065054  0.082617   
Beer (local and imported)             0.058637  0.027017  0.029038  0.093277   
Bread                                -0.033630  0.029886  0.035566  0.046609   
Brown beans                          -0.000968  0.056396  0.049499  0.080441   
Chicken                              -0.099556 -0.034328  0.028021  0.170077   
Chocolate drinks                      0.001434  0.034579  0.098319  0.052748   
Cocoyam                              -0.065245  0.084657  0.041852  0.004432   
Condiments,(salt,spices,pepper, etc)  0.086009  0.061731  0.032152  0.092006   
Fish-Dried                           -0.043431  0.063792  0.032209  0.041786   
Fish-Fresh                           -0.101088  0.028648  0.099715  0.094452   
Fish-Frozen                          -0.010361  0.010273  0.001762  0.030318   
Fish-Smoked                           0.022156  0.119212  0.021691  0.042435   
Garden eggs/egg plant                -0.029242  0.036982  0.026454  0.054099   
Gari -Yellow                         -0.115648 -0.044250  0.033370  0.025930   
Gari-White                           -0.064076  0.049750  0.056427  0.084982   
Goat                                 -0.050528 -0.044049  0.077087  0.025852   
Groundnut oil                         0.008431  0.079282  0.050751  0.060942   
Malt drinks                          -0.015759  0.034282  0.039556  0.062407   
Milk powder                           0.027711  0.012369  0.081368  0.070916   
Milk tinned (unsweetened)            -0.031031  0.036805  0.005204  0.019848   
Okra-fresh                            0.006330  0.043568  0.065175  0.032447   
Onions                               -0.011706  0.048467  0.048327  0.036509   
Orange/tangerine                      0.027295  0.027135  0.017173 -0.013428   
Other vegetables (fresh or canned)   -0.078398  0.007630  0.011609 -0.020285   
Palm oil                              0.006678  0.061643  0.024281  0.046470   
Pineapples                           -0.041482 -0.036824 -0.009721  0.083740   
Plantains                            -0.006457  0.099794  0.046588  0.112036   
Rice-Imported                        -0.061657  0.164859  0.116993  0.108241   
Rice-local                           -0.005615  0.093858  0.078141  0.048820   
Sachet water                         -0.124025  0.003404  0.005851  0.022638   
Seafood (lobster, crab, prawns)      -0.130993 -0.067230  0.072399 -0.017269   
Soft drinks (Coca cola, spirit etc)  -0.010483  0.050447  0.036618  0.033836   
Sugar                                 0.051525  0.089521  0.092739 -0.013307   
Sweet Potatoes                        0.042996  0.020629  0.064131  0.141176   
Tea                                  -0.056140  0.048430 -0.075744  0.071050   
Tomato puree(canned)                 -0.064994  0.067261  0.022496  0.052115   
Tomatoes                             -0.002188  0.034119  0.040328  0.060975   
White beans                          -0.019877  0.029185  0.059765  0.035741   
Yam-roots                            -0.036608  0.040986  0.051080  0.058166   

                                                                              \
k                                        F 4-8     F 51+    F 9-13     M 0-3   
i                                                                              
(Cocoyam, Spinach, etc)              -0.024821  0.029916 -0.018381 -0.053808   
Agricultural eggs                    -0.013403 -0.011610 -0.038304 -0.027416   
Bananas                              -0.

In [13]:
result.get_beta().to_dataframe().head()

,beta
i,
"(Cocoyam, Spinach, etc)",0.257887
Agricultural eggs,0.492479
Bananas,0.336030
Beef,0.317204
Beer (local and imported),0.333379


In [17]:
Products = list(x)

x_2010 = x[0:4826]
x_2012 = x[4827:9585]
x_2015 = x[9586:14165]
x_2018 = x[14166: 19142]

y_2010 = y[0:4826]
y_2012 = y[4827:9585]
y_2015 = y[9586:14165]
y_2018 = y[14166: 19142]

Factor = [0.5, 0.75, 1 , 1.5, 2 , 3, 4, 32]

In [20]:
variable_labels = {'SP.POP.3539.MA': 'Population'}
wb.get_dataframe(variable_labels, "NGA").loc[:, 'Population'].values[4]

5009609.0

In [21]:
nigeria_country_code = 'NGA'
nigeria_gdp_lcu = 'NY.GDP.MKTP.KN' 
years = [2010, 2012, 2015, 2018]



gdp_lcu_label = {nigeria_gdp_lcu :"GDP (LCU)"}
nigeria_gdp_lcu = wb.get_dataframe(gdp_lcu_label, country = "NGA").loc[:, "GDP (LCU)"].values[2019-2010]

In [22]:
average_df = dataframe_from_gspreadsheet("Nigeria - HH Characteristics", HH_and_Expenditure_Sheet['P3 HH/E Characteristics'])


GDP(LCU) is local currency units (not adjusted for exchange rate in USD)
inflation stayed relatively stable during this time
data taken from worldbank dank org


In [36]:
def product_expenditure(product, year):
    if year == 2010:
        data_frame = x_2010
    elif year == 2012:
        data_frame = x_2012
    elif year == 2015:
        data_frame = x_2015
    elif year == 2018:
        data_frame = x_2018
    elif year == 'All':
        data_frame = x
    product_sum = (data_frame[str(product)].sum())
    return product_sum

def product_total_expenditure(product):
    product_sum = (x[str(product)].sum())
    return product_sum

def expenditure_percent_change(product):
    final_value = product_expenditure(product, 2018)
    initial_value = product_expenditure(product, 2010)
    percent_change = ((final_value-initial_value)/initial_value) * 100
    return str(round(percent_change, 2)) + '%'

def percent_change_graph(product):
    product_2010 = product_expenditure(product, 2010)
    product_2012 = product_expenditure(product, 2012)
    product_2015 = product_expenditure(product, 2015)
    product_2018 = product_expenditure(product, 2018)
    point_1 = (round(((product_2012-product_2010)/product_2010) *100,2)) 
    point_2 = (round(((product_2015-product_2012)/product_2015) *100,2))
    point_3 = (round(((product_2018-product_2015)/product_2018) *100,2))
    years_array = ds.make_array(2012, 2015, 2018)
    product_array = ds.make_array(product, product, product)
    percent_change_array = ds.make_array(point_1, point_2, point_3)
    graph_df = pd.DataFrame({
        'Product':product_array, 
        'Year':years_array, 
        'Percent Change':percent_change_array
    })
    graph = px.line(graph_df, x = 'Year', y = 'Percent Change', title = str(product) + ' Expenditure Percent Chage')
    return graph.show()
def ln_graph(product):
    product_2010 = product_expenditure(product, 2010)
    product_2012 = product_expenditure(product, 2012)
    product_2015 = product_expenditure(product, 2015)
    product_2018 = product_expenditure(product, 2018)
    point_1 = np.log(product_2010)
    point_2 = np.log(product_2012)
    point_3 = np.log(product_2015)
    point_4 = np.log(product_2018)
    years_array = ds.make_array(2010, 2012, 2015, 2018)
    product_array = ds.make_array(product, product, product, product)
    percent_change_array = ds.make_array(point_1, point_2, point_3, point_4)
    graph_df = pd.DataFrame({
        'Product':product_array, 
        'Year':years_array, 
        'Log':percent_change_array
    })
    return px.line(graph_df, x = 'Year', y = 'Log', title = str(product) + ' Demand Log Change')

def gdp_per_capita_df():
    years_list = [2010, 2012, 2015, 2018]
    gdp_per_capita_list = [344549.92, 432649.56, 525444.83, 659159.14]
    inflation_rate_list = ['13.72%', '12.22%', '9.01%', '12.1%']
    gdp_per_capita_df = pd.DataFrame({
        'Year':years_list, 
        'GDP per Capita (LCU)':gdp_per_capita_list,
        'Inflation Rate':inflation_rate_list
    })
    return gdp_per_capita_df 

def gdp_per_capita_percent_change():
    final_value = gdp_per_capita_df().loc[:, 'GDP per Capita (LCU)'].values[3]
    initial_value = gdp_per_capita_df().loc[:, 'GDP per Capita (LCU)'].values[0]
    percent_change = ((final_value-initial_value)/initial_value) * 100
    return str(round(percent_change, 2)) + '%'

def gdp_per_capita_percent_change_graph():
    gdp_2010 =  gdp_per_capita_df().loc[:, 'GDP per Capita (LCU)'].values[0]
    gdp_2012 =  gdp_per_capita_df().loc[:, 'GDP per Capita (LCU)'].values[1]
    gdp_2015 =  gdp_per_capita_df().loc[:, 'GDP per Capita (LCU)'].values[2] 
    gdp_2018 =  gdp_per_capita_df().loc[:, 'GDP per Capita (LCU)'].values[3]
    point_1 = (round(((gdp_2012-gdp_2010)/gdp_2010) *100,2)) 
    point_2 = (round(((gdp_2015-gdp_2012)/gdp_2015) *100,2))
    point_3 = (round(((gdp_2018-gdp_2015)/gdp_2018) *100,2))
    years_array = ds.make_array(2012, 2015, 2018)
    gdp_array = ds.make_array(gdp_2012, gdp_2015, gdp_2018)
    percent_change_array = ds.make_array(point_1, point_2, point_3)
    gdp_graph_df = pd.DataFrame({
        'GDP per Capita (LCU)':gdp_array, 
        'Year':years_array, 
        'Percent Change':percent_change_array
    })
    return gdp_graph_df.plot.line(x = 'Year', y = 'Percent Change', title = 'GDP per Capita (LCU) Percent Chage')


def household_mean_df():
    girls_mean = ds.make_array(average_df['Girls'].mean())
    boys_mean = ds.make_array(average_df['Boys'].mean())
    women_mean = ds.make_array(average_df['Women'].mean())
    men_mean = ds.make_array(average_df['Men'].mean())
    average_hh_array = ds.make_array(girls_mean, boys_mean, women_mean, men_mean)
    average_hh_size = sum(average_hh_array)
    household_df = pd.DataFrame({
        'Average Girls': girls_mean,
        'Average Boys': boys_mean,
        'Average Women': women_mean,
        'Average Men': men_mean,
        'Average Household Size': average_hh_size
    })
    return household_df

def average_product_expenditure(product):
    product_average = (x[str(product)].mean())
    return product_average

def average_product_expenditure_per_person(product):
    product_average = (x[str(product)]).mean()
    household_average_size = int(household_mean_df()['Average Household Size']) + .72
    per_person_average_expenditure = product_average/household_average_size
    return per_person_average_expenditure

In [27]:
interact(lambda Products:
         print(("Average Household Expenditure: " + str(average_product_expenditure(product = Products)))), 
        Products=Dropdown(options=np.unique(Products)));

interactive(children=(Dropdown(description='Products', options=('(Cocoyam, Spinach, etc)', 'Agricultural eggs'…

In [28]:
interact(lambda Products:
         print(("Total Expenditure: $" + str(product_total_expenditure(product = Products)))), 
        Products=Dropdown(options=np.unique(Products)));

interactive(children=(Dropdown(description='Products', options=('(Cocoyam, Spinach, etc)', 'Agricultural eggs'…

In [29]:
interact(lambda Products:
         print(("Percent Change in Expenditure: " + str(expenditure_percent_change(product = Products)))), 
        Products=Dropdown(options=np.unique(Products)));

interactive(children=(Dropdown(description='Products', options=('(Cocoyam, Spinach, etc)', 'Agricultural eggs'…

In [30]:
interact(lambda Products:
         print(("Total Expenditure: $" + str(product_total_expenditure(product = Products)))), 
        Products=Dropdown(options=np.unique(Products)));

interact(lambda Products:
         print(("Percent Change: " + str(expenditure_percent_change(product = Products)))), 
        Products=Dropdown(options=np.unique(Products)));

interactive(children=(Dropdown(description='Products', options=('(Cocoyam, Spinach, etc)', 'Agricultural eggs'…

interactive(children=(Dropdown(description='Products', options=('(Cocoyam, Spinach, etc)', 'Agricultural eggs'…

In [37]:
interact(lambda Products:
         print((percent_change_graph(product = Products))), 
        Products=Dropdown(options=np.unique(Products)));

interactive(children=(Dropdown(description='Products', options=('(Cocoyam, Spinach, etc)', 'Agricultural eggs'…